In [ ]:
import os
import glob
import numpy as np

In [ ]:
def save_fsleyes_vis(Anat,Effect,Stat,Out,Threshold,task):
    
    #setting this, so that we can generate output even if treshold is inf (will still be set to inf in out file name)
    if np.isinf(Threshold):
        Threshold = 1000
    
    #effect overlay settings explanation: 
    
    #modulateRange specifies the range of z-score values that will be considered for modulating the color intensity of the effect size map overlay
    #values outside of this range will have the maximum/minimum modulation effect on the color intensity of the overlay
    #set to +/- 7 because that is also the surface figure cutoff for z scores
    
    #displayRange specifies that the color mapping for the effect size map should be based on the values in the range provided
    #values in the effect size map outside of the range may still exist, but they will be mapped to the minimum or maximum color in the specified colormaps
    #set to +/- 
    
    #don't include clippingRange since it would prevent values from the effect size map outide of this range from being displayed
    
    if task == 'mid':
        
        Out+='display-coronal.png'
        
        ! fsleyes render --neuroOrientation --outfile $Out -sz  500 500 \
           --selectedOverlay 1 --scene ortho --worldLoc 0 10 12 \
           --zcentre  0  0 --showLocation no \
           --layout horizontal --hidex --hidez --hideCursor \
           --showColourBar --colourBarLocation right --colourBarLabelSide top-left --colourBarSize 65.0 --labelSize 14 \
           $Anat --overlayType volume  \
           $Effect --name "" --overlayType volume --modulateImage $Stat --cmap yellow --negativeCmap green --useNegativeCmap --displayRange 0.0 13 --modulateRange 0.0 7.0 --modulateAlpha \
           $Stat --overlayType mask --alpha 100.0 --threshold -$Threshold $Threshold --outline --maskColour 0 0 0
    
    else:
        
        Out+='display-axial.png'
        
        ! fsleyes render --neuroOrientation --outfile $Out -sz  500 500 \
           --selectedOverlay 1 --scene ortho --worldLoc 0 0 0 \
           --zcentre  0  0 --showLocation no \
           --layout horizontal --hidex --hidey --hideCursor \
           --showColourBar --colourBarLocation right --colourBarLabelSide top-left --colourBarSize 85.0 --labelSize 14 \
           $Anat --overlayType volume  \
           $Effect --name "" --overlayType volume --modulateImage $Stat --cmap yellow --negativeCmap green --useNegativeCmap --displayRange 0.0 13 --modulateRange 0.0 7.0 --modulateAlpha \
           $Stat --overlayType mask --alpha 100.0 --threshold -$Threshold $Threshold --outline --maskColour 0 0 0
    
    return

In [ ]:
def get_files(group,ses,task,contrast):
    
    if len(glob.glob(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_std_effect_size.nii.gz')) == 0:
        print('No effect size maps, so cannot generate fsleyes output for: '+' '.join([group,ses,task,contrast]))
        return (None, None, None, None, None)

    
    #get anatfile, statmap and effectmap
    anat_file='../templates/tpl-MNI152NLin6Asym_res-01_desc-brain_T1w.nii.gz'
    effect_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_std_effect_size.nii.gz'
    stat_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_z_score.nii.gz'

    #get threshold
    threshold_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_FDR_threshold.txt'
    with open(threshold_file) as f:
        threshold = float(f.readline().strip('\n'))
    rounded_threshold = round(threshold,2)

    #get participant count
    participants_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_participants.txt'
    with open(participants_file) as f:
        part_count = len(f.readlines())

    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_volume','visualization','fsleyes_indiv_figures',f'task-{task}')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    #make outfile path
    #ending will be adding depending if axial or coronal
    out_file = f'../../../derivatives/task_analysis_volume/visualization/fsleyes_indiv_figures/task-{task}/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_threshold}_n-{part_count}_'

    if threshold > 100:
        print('Threshold is infinity for: '+' '.join([group,ses,task,contrast]))
        
        #if commented out, will prevent png with inf as threshold from being saved
#         return (None, None, None, None, None)

    return (anat_file,effect_file,stat_file,out_file,threshold)

In [ ]:
task='mid'
sessions = ['baseline','1year']
groups = ['HC','MM']


if task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','UnsuccStop-SuccStop']
    
elif task == 'nback':
    contrasts=['twoback-zeroback']

elif task == 'mid':
    contrasts = ['HiRewCue-NeuCue', #high reward anticipation -- paper A2, ABCD
                          'HiRewCue-LoRewCue', #high vs. low reward anticipation -- paper A3, ABCD
                          'RewCue-NeuCue', #combined reward anticipation -- paper A1
                          'LoRewCue-NeuCue', #low reward anticipation -- ABCD
                          
                          'HiLossCue-NeuCue', #high loss anticipation -- paper A5, ABCD
                          'HiLossCue-LoLossCue', #high vs. low loss anticipation -- ABCD
                          'LossCue-NeuCue', #combined loss anticipation
                          'LoLossCue-NeuCue', #low loss anticipation -- ABCD

                          'HiRewCue-HiLossCue', #high reward vs. high loss anticipation
                          'RewCue-LossCue', #combined reward vs. loss anticipation
                 
                          'HiRewCue-Baseline', #high reward anticipation vs. baseline -- paper A4
             
                          'HiWin-NeuHit', #high reward outcome cp. to neutral hit -- paper O6
                          'Win-NeuHit', #combined reward outcome cp. to neutral hit
                          
                          'HiWin-HiNoWin', #high reward outcome cp. to high reward miss
                          'Win-NoWin', #combined reward outcome cp. to combined reward miss -- ABCD
                          
                          'HiLoss-NeuMiss', #high loss cp. to neutral miss
                          'Loss-NeuMiss', #combined loss cp. to neutral miss
                          
                          'HiLoss-AvoidHiLoss', #high loss cp. to high avoid loss
                          'Loss-AvoidLoss', #combined loss cp. to combined avoid loss -- ABCD
                          
                          'HiLoss-NeuHit', #high loss cp. to neutral hit -- paper O7
                          'Loss-NeuHit', #combined loss cp. to neutral hit

                          'HiWin-HiLoss', #high reward outcome cp. to high loss
                          'Win-Loss', #combined reward outcome cp. to combined loss
                         ]
    
    
for ses in sessions:
    for group in groups:        
        for contrast in contrasts:
            anat_file,effect_file,stat_file,out_file,threshold = get_files(group,ses,task,contrast)
            
            #only call if input found 
            if effect_file:
                save_fsleyes_vis(anat_file,effect_file,stat_file,out_file,threshold,task)
                